#EXEMPLO PRÁTICO

##Suposição de Diagnóstico Clínico com Modelo BERT

💻 [acessar este jupyter notebook no colab](https://colab.research.google.com/drive/17BhS8-GnGkZk7tzTL23kQHoBSADRpeVC?usp=sharing)

Este notebook expõe um exemplo prático de aplicação de Processamento de Linguagem Natural (NLP) e do uso de um Modelo de Linguagem de Grande Escala (LLM) em uma solução de chatbot simples. O projeto visa treinar o modelo BERT através de fine-tunning (ajuste de modelo especializado) para reconhecer sintomas de doenças como entidades nomeadas (NER) e retornar um possível diagnóstico através de um texto de apoio.


**Exemplo:**

```
O paciente José relata tontura e visão turva há quatro dias.
```

> As entidades nomeadas serão os sintomas: **"tontura"** e **"visão turva"**


A partir dessa extração dos sintomas, será retornado um possível diagnóstico ao paciente usando um LLM da Hugging Face.



##Configurações do ambiente

A biblioteca spaCy trabalha com modelos de idiomas já pré-treinados. É necessário fazer o download de algum desses modelos, pois eles contêm informações sobre linguagens, vocabulários, vetores treinados, sintaxes e entidades.

* O modelo spaCy a ser utilizado neste projeto, será o modelo pré-treinado para o português: `pt_core_news_lg` é a versão "large" (grande) do modelo, que possui 550 MB e é mais preciso do que versões menores.

In [5]:
!pip install -U spacy==3.2.0 -q

#baixar modelo pré-treinado em português (lg é a versão large de 550MB)
!python -m spacy download 'pt_core_news_lg'

2024-08-02 14:01:32.679229: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-02 14:01:32.703829: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-02 14:01:32.710821: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-02 14:01:34.243089: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
DEPRECATION: https://github.com/explosion/spacy-models/releases/download/pt_core_news_lg-3.2.0/pt_core_news_lg-3.2.0-py3-none-any.whl#egg=pt_core_news_lg==3.2.0 contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this beha

In [6]:
import spacy
import pt_core_news_lg

#cria um objeto para usar o modelo pré-treinado
nlp = spacy.load('pt_core_news_lg')

/usr/local/lib/python3.10/dist-packages/torch/__init__.py:749: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:431.)
  _C._set_default_tensor_type(t)


##Leitura de entidades nomeadas

In [8]:
texto_exemplo = nlp("Maria trabalha na Google")

for entidade in texto_exemplo.ents:
  print(entidade.text, entidade.label_)

Maria PER
Google ORG


In [9]:
#destacando visualmente
spacy.displacy.render(texto_exemplo, style='ent', jupyter=True)

In [10]:
#visualizando dependência na frase
spacy.displacy.render(texto_exemplo, style="dep", jupyter=True,
                options={'compact': True, 'distance':100, 'color': '#493082', 'bg': '#56D8FD', 'font': 'Arial'})

###Realizando a leitura de entidades nomeadas desejadas

Em português existem poucas entidades nomeadas no modelo pré-treinado da biblioteca spaCy. Portanto é necessário inserir dados para realizar um pré-treino do algoritmo com novas labels e melhorar o modelo.

In [11]:
texto_exemplo = nlp("O paciente José relata tontura e visão turva há quatro dias.")
spacy.displacy.render(texto_exemplo, style='ent', jupyter=True)

In [12]:
for entidade in texto_exemplo.ents:
  print(entidade.text, entidade.label_)

José PER


####Fine-Tuning (ajuste de modelo especializado)

Como não existe a rotulação de entidade nomeada para sintomas no modelo em português, é necessário realizar a tarefa de fine-tunning.

In [14]:
TRAIN_DATA = [
    ("O paciente se queixa de dor no peito e falta de ar por três dias seguidos.",
     {"entities": [(24, 35, "SYMPTOM"), (38, 48, "SYMPTOM")]}),
    ("Reclamação de fraqueza no corpo e tosse seca por uma semana.",
     {"entities": [(16, 29, "SYMPTOM"), (33, 43, "SYMPTOM")]}),
    ("Paciente apresenta febre alta e dor de cabeça há dois dias.",
     {"entities": [(17, 27, "SYMPTOM"), (30, 44, "SYMPTOM")]}),
    ("Relata dor abdominal intensa e náusea constante há quatro dias.",
     {"entities": [(7, 23, "SYMPTOM"), (27, 43, "SYMPTOM")]}),
    ("Queixa de dor nas costas e dificuldade para respirar por uma semana.",
     {"entities": [(10, 23, "SYMPTOM"), (27, 50, "SYMPTOM")]}),
    ("Paciente com dor nas articulações e febre baixa há três dias.",
     {"entities": [(10, 29, "SYMPTOM"), (33, 44, "SYMPTOM")]}),
    ("Queixa de dor muscular e cansaço extremo há cinco dias.",
     {"entities": [(10, 22, "SYMPTOM"), (25, 39, "SYMPTOM")]}),
    ("Dor de garganta e dificuldade para engolir por dois dias.",
     {"entities": [(0, 14, "SYMPTOM"), (18, 39, "SYMPTOM")]}),
    ("Paciente relata tontura e visão turva há quatro dias.",
     {"entities": [(16, 23, "SYMPTOM"), (27, 38, "SYMPTOM")]}),
    ("Reclamação de dor nas pernas e inchaço nos tornozelos por uma semana.",
     {"entities": [(16, 29, "SYMPTOM"), (33, 52, "SYMPTOM")]}),
    ("Queixa de dor de ouvido e secreção purulenta há dois dias.",
     {"entities": [(10, 23, "SYMPTOM"), (27, 44, "SYMPTOM")]}),
    ("Paciente apresenta falta de apetite e perda de peso há uma semana.",
     {"entities": [(17, 33, "SYMPTOM"), (36, 49, "SYMPTOM")]}),
    ("Relata dor no joelho e dificuldade para caminhar por três dias.",
     {"entities": [(7, 20, "SYMPTOM"), (24, 45, "SYMPTOM")]}),
    ("Queixa de dor no ombro e rigidez no pescoço há cinco dias.",
     {"entities": [(10, 23, "SYMPTOM"), (27, 44, "SYMPTOM")]}),
    ("Paciente com dor nas costas e febre há dois dias.",
     {"entities": [(10, 23, "SYMPTOM"), (27, 32, "SYMPTOM")]}),
    ("Reclamação de dor no peito e falta de ar há quatro dias.",
     {"entities": [(16, 28, "SYMPTOM"), (32, 42, "SYMPTOM")]}),
    ("Queixa de dor nas costas e dificuldade para respirar há uma semana.",
     {"entities": [(10, 23, "SYMPTOM"), (27, 50, "SYMPTOM")]}),
    ("Paciente apresenta dor de cabeça e tontura há três dias.",
     {"entities": [(17, 29, "SYMPTOM"), (33, 40, "SYMPTOM")]}),
    ("Relata dor abdominal e náusea constante há cinco dias.",
     {"entities": [(7, 21, "SYMPTOM"), (25, 39, "SYMPTOM")]}),
    ("Queixa de dor nas articulações e febre baixa há quatro dias.",
     {"entities": [(10, 29, "SYMPTOM"), (33, 44, "SYMPTOM")]}),
    ("Paciente com dor muscular e cansaço extremo há uma semana.",
     {"entities": [(10, 22, "SYMPTOM"), (25, 39, "SYMPTOM")]}),
    ("Queixa de dor de garganta e dificuldade para engolir há dois dias.",
     {"entities": [(10, 24, "SYMPTOM"), (28, 49, "SYMPTOM")]}),
    ("Paciente relata dor de ouvido e secreção purulenta há três dias.",
     {"entities": [(17, 30, "SYMPTOM"), (34, 51, "SYMPTOM")]}),
    ("Reclamação de falta de apetite e perda de peso há quatro dias.",
     {"entities": [(16, 32, "SYMPTOM"), (35, 48, "SYMPTOM")]}),
    ("Queixa de dor no joelho e dificuldade para caminhar há uma semana.",
     {"entities": [(10, 23, "SYMPTOM"), (27, 48, "SYMPTOM")]}),
    ("Paciente apresenta dor no ombro e rigidez no pescoço há dois dias.",
     {"entities": [(17, 30, "SYMPTOM"), (34, 51, "SYMPTOM")]}),
    ("Relata dor nas costas e febre baixa há três dias.",
     {"entities": [(7, 20, "SYMPTOM"), (24, 35, "SYMPTOM")]}),
    ("Queixa de dor abdominal e náusea constante há cinco dias.",
     {"entities": [(10, 24, "SYMPTOM"), (28, 42, "SYMPTOM")]}),
    ("Paciente com dor de cabeça e tontura há quatro dias.",
     {"entities": [(10, 22, "SYMPTOM"), (26, 33, "SYMPTOM")]}),
    ("Reclamação de dor nas articulações e febre alta há uma semana.",
     {"entities": [(16, 35, "SYMPTOM"), (39, 49, "SYMPTOM")]}),
    ("Queixa de dor muscular e cansaço extremo há três dias.",
     {"entities": [(10, 22, "SYMPTOM"), (25, 39, "SYMPTOM")]}),
    ("Paciente relata dor de garganta e dificuldade para engolir há cinco dias.",
     {"entities": [(17, 31, "SYMPTOM"), (35, 56, "SYMPTOM")]}),
    ("Relata dor no peito e falta de ar há dois dias.",
     {"entities": [(7, 19, "SYMPTOM"), (23, 33, "SYMPTOM")]}),
    ("Queixa de dor nas costas e dificuldade para respirar há quatro dias.",
     {"entities": [(10, 23, "SYMPTOM"), (27, 50, "SYMPTOM")]}),
    ("Paciente com dor no joelho e inchaço nos tornozelos há três dias.",
     {"entities": [(10, 23, "SYMPTOM"), (27, 46, "SYMPTOM")]}),
    ("Reclamação de dor nas costas e febre baixa há cinco dias.",
     {"entities": [(16, 29, "SYMPTOM"), (33, 44, "SYMPTOM")]}),
    ("Queixa de dor no ombro e rigidez no pescoço há uma semana.",
     {"entities": [(10, 23, "SYMPTOM"), (27, 44, "SYMPTOM")]}),
    ("Paciente relata dor muscular e cansaço extremo há dois dias.",
     {"entities": [(17, 29, "SYMPTOM"), (33, 47, "SYMPTOM")]}),
    ("Relata dor no peito e falta de ar há uma semana.",
     {"entities": [(7, 19, "SYMPTOM"), (23, 33, "SYMPTOM")]}),
]

Utiliza a função de treino para treinar um modelo de reconhecimento de entidades nomeadas (NER) usando a biblioteca SpaCy.

In [16]:
#FUNÇÃO DE TREINO
from spacy.training import Example
import random

def train_spacy(data, n_iter):
    TRAIN_DATA = data
    """Carrega o modelo, configura o pipeline e treina o reconhecedor de entidade."""
    nlp = spacy.blank('pt')
    print("Modelo Carregado")

    #criando o componente 'ner' e adicionando ao pipeline
    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe("ner", last=True)
    else:
        ner = nlp.get_pipe("ner")

    #adicionando as labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    #obtendo os nomes de outros pipes para desativá-los durante o treinamento
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):
        spacy.util.fix_random_seed()
        optimizer = nlp.begin_training()

        for itn in range(n_iter):
            random.seed(10)
            random.shuffle(TRAIN_DATA)
            losses = {}

            #criando uma lista de objetos Example para o treinamento
            examples = []
            for text, annotations in TRAIN_DATA:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                examples.append(example)

            #atualizando o modelo com os exemplos
            nlp.update(
                examples,
                drop=0.2,
                sgd=optimizer,
                losses=losses,
            )
            print("Losses", losses)

        for text, _ in TRAIN_DATA:
            doc = nlp(text)
            print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
            print("Tokens", [(t.text, t.ent_type_) for t in doc])
            print("\n")
    return nlp

In [17]:
nlp_ = train_spacy(data=TRAIN_DATA, n_iter=100)

Modelo Carregado


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "Paciente apresenta dor de cabeça e tontura há três..." with entities "[(17, 29, 'SYMPTOM'), (33, 40, 'SYMPTOM')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "Reclamação de dor nas pernas e inchaço nos tornoze..." with entities "[(16, 29, 'SYMPTOM'), (33, 52, 'SYMPTOM')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "P

Losses {'ner': 258.6666579246521}
Losses {'ner': 244.79598724842072}
Losses {'ner': 227.22244042158127}
Losses {'ner': 205.76503425836563}
Losses {'ner': 172.71656334400177}
Losses {'ner': 126.22295197844505}
Losses {'ner': 82.9011414796114}
Losses {'ner': 43.30264101177454}
Losses {'ner': 22.668253073468804}
Losses {'ner': 18.392712071014103}
Losses {'ner': 18.613640209759524}
Losses {'ner': 19.146797617394952}
Losses {'ner': 19.37737724739128}
Losses {'ner': 19.332347415743868}
Losses {'ner': 18.628898488039056}
Losses {'ner': 17.406204285119202}
Losses {'ner': 16.48604511819667}
Losses {'ner': 12.515881849842117}
Losses {'ner': 25.626112584639486}
Losses {'ner': 17.69955537658825}
Losses {'ner': 50.66192865226185}
Losses {'ner': 52.57764075437444}
Losses {'ner': 84.99315311573446}
Losses {'ner': 78.21466926066205}
Losses {'ner': 74.80211818637326}
Losses {'ner': 81.98779043741524}
Losses {'ner': 70.77452731691301}
Losses {'ner': 63.65461594611406}
Losses {'ner': 52.462135922629386}


In [18]:
text = nlp_("O paciente José relata tontura e visão turva há quatro dias.")
#text = nlp_("Paciente apresenta falta de apetite e perda de peso há uma semana.")
#text = nlp_("Queixa de dor nas costas e dificuldade para respirar há uma semana.")
#text = nlp_("Queixa de dor de garganta e dificuldade para engolir há dois dias.")
spacy.displacy.render(text, style='ent', jupyter=True)

In [19]:
for entidade in text.ents:
  print(entidade.text, entidade.label_)

tontura SYMPTOM
visão turva SYMPTOM


In [20]:
entidades = [entidade.text for entidade in text.ents]

if len(entidades) == 1:
    pergunta = f"Qual o possível diagnóstico para {entidades[0]}?"
elif len(entidades) >= 2:
    pergunta = f"Qual o possível diagnóstico para {entidades[0]} e {entidades[1]}?"

print(pergunta)


Qual o possível diagnóstico para tontura e visão turva?




---



##Perguntas e Respostas com modelo BERT

BERT (Bidirectional Encoder Representations from Transformers) é um modelo de linguagem desenvolvido pelo Google que revolucionou o processamento de linguagem natural desde seu lançamento. Para utilizar BERT e outros modelos relacionados é só acessar através da biblioteca transformers da Hugging Face.

Os passos seguintes são para responder a pergunta que foi gerada na etapa anterior.

In [21]:
!pip install transformers

In [22]:
import transformers
from transformers import pipeline

In [23]:
qea = pipeline("question-answering", model="pierreguillou/bert-base-cased-squad-v1.1-portuguese")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/862 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/494 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [24]:
texto = """Dor no peito e falta de ar podem indicar condições cardíacas, como angina ou infarto do miocárdio, e também doenças respiratórias, como pneumonia ou bronquite.

          Fraqueza no corpo e tosse seca podem sugerir infecções virais, como gripe, ou doenças mais graves, como tuberculose.

          Febre alta e dor de cabeça são comuns em infecções, incluindo gripes e resfriados, mas também podem estar presentes em condições sérias como meningite ou sinusite.

          Dor abdominal intensa e náusea constante são frequentemente associados a problemas gastrointestinais, como gastrite ou úlceras.

          Dor nas costas e dificuldade para respirar podem ser sinais de condições que afetam a coluna vertebral ou os pulmões, como pneumonia, doenças pulmonares crônicas ou condições musculoesqueléticas, como uma hérnia de disco.

          Dor nas articulações e febre baixa podem ocorrer em doenças inflamatórias, como artrite reumatoide, ou outras condições autoimunes.

          Dor muscular e cansaço extremo podem ser associados a esforço físico excessivo ou a condições como fibromialgia ou síndrome de fadiga crônica.

          Dor de garganta e dificuldade para engolir são frequentemente sintomas de infecções na garganta, como faringite ou amigdalite.

          Tontura e visão turva podem estar relacionadas a problemas de pressão arterial ou distúrbios neurológicos.

          Dor nas pernas e inchaço nos tornozelos podem ser sintomas de problemas circulatórios, como insuficiência venosa ou trombose venosa profunda.

          Dor de ouvido e secreção purulenta são sinais de infecção no ouvido, como otite.

          Falta de apetite e perda de peso podem indicar problemas gastrointestinais ou doenças mais graves, como câncer.

          Dor no joelho e dificuldade para caminhar podem ser indicativas de problemas articulares, como artrite ou lesões.

          Dor no ombro e rigidez no pescoço são frequentemente associadas a tensões musculares ou problemas cervicais.

          Dor nas costas e febre podem estar presentes em infecções renais ou problemas na coluna vertebral.

          Dor abdominal e náusea constante podem indicar uma infecção gastrointestinal ou problemas digestivos.

          Dor de cabeça e tontura podem estar associadas a condições neurológicas ou problemas de pressão arterial.

          A combinação e persistência desses sintomas fornecem pistas valiosas para possíveis condições médicas, e uma avaliação médica é essencial para um diagnóstico preciso e tratamento adequado."""

In [25]:
resposta = qea(question=pergunta, context=texto)

print("Pergunta:", pergunta)
print("Resposta: Suposto quadro de", resposta['answer'])
print("Score: ", resposta['score'])

Pergunta: Qual o possível diagnóstico para tontura e visão turva?
Resposta: Suposto quadro de problemas de pressão arterial ou distúrbios neurológicos
Score:  0.636976957321167
